Modelo de Recomendacion

In [16]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [9]:
archivo =  'df_reviews_api.csv'
df = pd.read_csv(archivo)
df.head(1)

,Unnamed: 0,user_id,user_url,item_id,helpful,recommend,sentiment_analysis
0,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,730,0 of 1 people (0%) found this review helpful,True,1


In [53]:
# Seleccionar columnas mas relevantes
df_user = df[['user_id','item_id','recommend','sentiment_analysis']]
df_user.head(10)

,user_id,item_id,recommend,sentiment_analysis
0,76561198079601835,730,True,1
1,MeaTCompany,730,True,2
2,76561198089393905,72850,True,0
3,76561198089393905,440,True,1
4,76561198156664158,252950,True,2
5,76561198077246154,440,True,1
6,76561198077246154,304930,True,1
7,WeiEDKrSat,4000,True,1
8,thequeenpanda,42910,True,1
9,Rainbow-Dashie,730,True,2


In [47]:
class ModeloRecomendacion:
    def __init__(self, df_user):
        self.df_user = df_user
        self.model = NearestNeighbors(metric='cosine', algorithm='brute')
        self.user_item_matrix = self._create_user_item_matrix()
        self.model.fit(self.user_item_matrix)
    
    def _create_user_item_matrix(self):
        user_item_matrix = df_user.pivot_table(index='user_id', columns='item_id', values='sentiment_analysis', fill_value=0)
        return user_item_matrix
    
    def recomendacion_usuario(self, user_id):
        user_index = self.user_item_matrix.index.get_loc(user_id)
        distances, indices = self.model.kneighbors(self.user_item_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=6)
        similar_users = self.user_item_matrix.index[indices.flatten()]
        recommended_games = []
        for user in similar_users[1:]:
            recommended_games.extend(self.df_user[(self.df_user['user_id'] == user) & (self.df_user['recommend'] == True) & (~self.df_user['item_id'].isin(recommended_games))]['item_id'].tolist())
        return recommended_games[:5]

# Crear una instancia del modelo de recomendación
modelo = ModeloRecomendacion(df_user)

In [48]:
# Obtener recomendaciones para un usuario específico
user_id = 'zombi_anon'
recomendaciones = modelo.recomendacion_usuario(user_id)
print(recomendaciones)

[304050, 218230, 252490, 322330, 242880]


In [49]:
# Obtener recomendaciones para un usuario específico
user_id = 'zombieskiler6969'
recomendaciones = modelo.recomendacion_usuario(user_id)
print(recomendaciones)

[345180, 359550, 32800, 323500, 50130]
